<a href="https://colab.research.google.com/github/Knockjoy/browse-on-google-colab/blob/main/browser10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下準備

In [75]:
#@title settings
port = "8000"#@param {type:"string"}
host = True #@param {type:"boolean"}

In [ ]:
#@title shell
%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg
Package: *
Pin: release a=eoan
Pin-Priority: 500
Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300
Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF
apt-get update
apt-get install chromium chromium-driver
pip install selenium
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.4.7.4
pip install flask

In [77]:
#@title create browser.py
browser_py='''
from selenium_profiles.utils.installer import install_chromedriver
from selenium_profiles.driver import driver as mydriver
from selenium_profiles.profiles import profiles
from selenium.webdriver.common.by import By
from selenium_profiles.utils.colab_utils import display, showscreen, show_html 
class Brows():
  def startup(self):
    install_chromedriver()
    self.mydriver = mydriver()
    self.display = display()
    self.display.start_display()
    self.driver = self.mydriver.start(profiles.Windows(), uc_driver=False)  # or .Android    
  def access(self,url):
    print('access now')
    self.driver.get(url)
    self.save_screen('browser/static/screen.png')
    # showscreen(self.driver)
  def save_screen(self,path):
    self.driver.save_screenshot(path)
    print('saved')
  def exit(self):
    self.driver.quit()
    self.display.stop_display()
  def search_ongoogle(self,word):
    self.access(f'https://google.com/search?q={word}')
'''.strip()


In [78]:
# from browser import browser
# chrome_browser = browser.Brows()
# chrome_browser.startup()
# chrome_browser.access("https://twitter.com")
# chrome_browser.exit()

In [79]:
project_name="browser"
from google.colab.output import eval_js
host = eval_js(f"google.colab.kernel.proxyPort({str(port)})")

In [125]:
#@title app.py
app_py_string = '''
import sys
import browser
from flask import Flask, redirect ,request,render_template,jsonify
import json
import requests

class ctrl_driver:
  def driver_run(self):
    self.chrome_browser = browser.Brows()
    self.chrome_browser.startup()
    self.chrome_browser.access("https://twitter.com")
  def driver_exit(self):
    self.chrome_browser.exit()

app = Flask(__name__)

@app.route("/")
def check():
    return render_template("index.html")

@app.route("/update")
def update():
  try:
    drive.chrome_browser.save_screen("browser/static/screen.png")
    return {"states":200,"msg":"ok"}
  except Exception:
    return {"states":500,"msg":"error"}
@app.route("/google",methods=["GET"])
def seachongoogle():
  try:
    word= request.args.get("word")
    if word is not None:
      drive.chrome_browser.search_ongoogle(word)
      return {"stats":200,"msg":"ok"}
    else:
      return {"stats":500,"msg":"word is None."}
  except:
    return {"stats":500,"msg":"error"}
@app.route("/access",methods=["GET"])
def access_web():
  try:
    url=request.args.get("url")
    if url is not None:
      drive.chrome_browser.access(url)
      return {"stats":200,"msg":"ok"}
    else :
      return {"stats":500,"msg":"url is None."}
  except:
      return {"stats":500,"msg":"error"}

# @app.route("/output", methods=["POST"])
# def output():
#     try:
#         _A = int(request.json["_A"])
#         _B = int(request.json["_B"])
#         res = _A + _B
#         data = {"result":res, "_A":_A, "_B":_B, "msg":"%s + %s = %s"%(_A,_B,res)}
#         return jsonify(ResultSet=json.dumps(data)), 200
#     except Exception as err:
#         print(err)
#         data = {"error":"err"}
#         return jsonify(ResultSet=json.dumps(data)), 500

if __name__ == "__main__":
  import sys
  args = sys.argv
  drive=ctrl_driver()
  drive.driver_run()
  app.run(debug=True, port=args[1],use_reloader=False)
'''.strip()

In [126]:
#@title index.html
index_html_string = '''

<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title>Document</title>
</head>
<body>
<canvas id="board" width="1024" height="648"></canvas>
    <script src="static/post.js"></script>
</body>
</html>

'''.strip()

In [133]:
#@title post.js
post_js_string='''

window.onload = ()=>{
  // canvas準備
  const board = document.querySelector("#board");  //getElementById()等でも可。オブジェクトが取れれば良い。
  const ctx = board.getContext("2d");

  // 画像読み込み
  const chara = new Image();
  chara.src = "static/screen.png";  // 画像のURLを指定
  chara.onload = () => {
    ctx.drawImage(chara, 0, 0);
  };
};

function draw_img(){
    // canvas準備
  const board = document.querySelector("#board");  //getElementById()等でも可。オブジェクトが取れれば良い。
  const ctx = board.getContext("2d");

  // 画像読み込み
  const chara = new Image();
  chara.src = "static/screen.png";  // 画像のURLを指定
  chara.onload = () => {
    ctx.drawImage(chara, 0, 0);
  };
}

function update_img(){
  fetch("/update")
  .then((response) => response.json())
    .then((data) => {
draw_img()
    });

}

function serchongoogle(word){
  fetch("/google?word="+word)
  .then((response) => response.json())
    .then((data) => {
draw_img()
    });

}

function access(url){
  fetch("/access?url="+url)
  .then((response) => response.json())
    .then((data) => {
draw_img()
    });
}

'''.strip()


In [134]:
#@title create project
!rm -rf $project_name
!mkdir $project_name && mkdir $project_name/templates && mkdir $project_name/static
!echo -e '{app_py_string}' >> $project_name/app.py
!echo -e '{index_html_string}' >> $project_name/templates/index.html
!echo -e '{post_js_string}' >> $project_name/static/post.js
!echo -e "{browser_py}">> $project_name/browser.py

In [ ]:
print('①が起動してから、↓のurlにアクセスする')
print(host)
#①
!python $project_name/app.py $port